In [53]:
import argparse
import logging
import os
import sys
from datetime import datetime, timedelta
import openstack
import psycopg2
from psycopg2.extras import RealDictCursor
from typing import Dict, List, Any
from dotenv import load_dotenv
from keystoneauth1.identity import v3
from keystoneauth1 import session
from openstack import connection
from tabulate import tabulate



In [54]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('resource_cleanup.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [55]:
import pytz

est = pytz.timezone('America/New_York')

# January 26th, 2025 8:00 PM EST
cutoff_time = datetime(2025, 1, 28, 20, 45, 0, tzinfo=est)

# Convert to UTC for database query since it's best practice to store timestamps in UTC
cutoff_time = cutoff_time.astimezone(pytz.UTC)

In [56]:
print(cutoff_time)

2025-01-29 01:41:00+00:00


In [57]:
load_dotenv()

True

In [5]:
db_params = {
            'dbname': os.getenv('DB_NAME'),
            'user': os.getenv('DB_USER'),
            'password': os.getenv('DB_PASSWORD'),
            'host': os.getenv('DB_HOST'),
            'port': os.getenv('DB_PORT')
        }


auth = v3.ApplicationCredential(
            auth_url=os.getenv('OS_AUTH_URL'),
            application_credential_id=os.getenv('OS_APPLICATION_CREDENTIAL_ID'),
            application_credential_secret=os.getenv('OS_APPLICATION_CREDENTIAL_SECRET')
        )
os_sess = session.Session(auth=auth)
os_conn = connection.Connection(session=os_sess)

In [19]:
def get_resources_to_delete() -> Dict[str, List[Dict]]:
        """Get resources older than specified hours that are still active"""

        # Calculating cut off time as (current timestamp - no of hours given as argument)
        est = pytz.timezone('America/New_York')

        # January 26th, 2025 8:00 PM EST
        cutoff_time = datetime(2025, 1, 27, 20, 0, 0, tzinfo=est)
        
        # Convert to UTC for database query since it's best practice to store timestamps in UTC
        cutoff_time = cutoff_time.astimezone(pytz.UTC)
        
        try:
            with psycopg2.connect(**db_params) as conn:
                with conn.cursor(cursor_factory=RealDictCursor) as cur:
                    resources = {}
                    
                    # Servers query
                    cur.execute("""
                        SELECT 
                            resource_id, resource_name, status, created_time,
                            updated_time, last_seen_time, first_time_not_seen,
                            flavor, image, security_groups, addresses
                        FROM servers
                        WHERE created_time > %s
                        AND first_time_not_seen IS NULL
                        ORDER BY created_time ASC
                    """, (cutoff_time,))
                    resources['servers'] = cur.fetchall()
                    
                    # Networks query
                    cur.execute("""
                        SELECT 
                            resource_id, resource_name, status, created_time,
                            updated_time, last_seen_time, first_time_not_seen,
                            port_security_enabled
                        FROM networks
                        WHERE created_time > %s
                        AND first_time_not_seen IS NULL
                        ORDER BY created_time ASC
                    """, (cutoff_time,))
                    resources['networks'] = cur.fetchall()
                    
                    # Routers query
                    cur.execute("""
                        SELECT 
                            resource_id, resource_name, status, created_time,
                            updated_time, last_seen_time, first_time_not_seen,
                            external_gateway_info
                        FROM routers
                        WHERE created_time > %s
                        AND first_time_not_seen IS NULL
                        ORDER BY created_time ASC
                    """, (cutoff_time,))
                    resources['routers'] = cur.fetchall()
                    
                    # Subnets query
                    cur.execute("""
                        SELECT 
                            resource_id, resource_name, status, created_time,
                            updated_time, last_seen_time, first_time_not_seen,
                            network_id, allocation_pools, cidr
                        FROM subnets
                        WHERE created_time > %s
                        AND first_time_not_seen IS NULL
                        ORDER BY created_time ASC
                    """, (cutoff_time,))
                    resources['subnets'] = cur.fetchall()
                        
            return resources
            
        except Exception as e:
            logger.error(f"Database error: {str(e)}")
            raise

In [20]:
def display_resources(resources: Dict[str, List[Dict]]):
        """Display resources that will be deleted"""
        for resource_type, items in resources.items():
            if items:
                print(f"\n{resource_type.upper()} to be deleted:")
                table_data = []
                for item in items:
                    age = datetime.now() - item['created_time']
                    row = [
                        item['resource_name'],
                        item['status'],
                        item['created_time'].strftime('%Y-%m-%d %H:%M:%S'),
                        f"{age.days}d {age.seconds//3600}h",
                        item['last_seen_time'].strftime('%Y-%m-%d %H:%M:%S')
                    ]
                    
                    # Add resource-specific information
                    if resource_type == 'servers':
                        row.append(f"Flavor: {item['flavor']}")
                    elif resource_type == 'subnets':
                        row.append(f"CIDR: {item['cidr']}")
                    
                    table_data.append(row)

                headers = ['Name', 'Status', 'Created', 'Age', 'Last Seen', 'Details']
                print(tabulate(table_data, headers=headers, tablefmt='grid'))

In [58]:
display_resources(get_resources_to_delete())


SERVERS to be deleted:
+----------------+----------+---------------------+-------+---------------------+-------------------+
| Name           | Status   | Created             | Age   | Last Seen           | Details           |
+================+==========+=====================+=======+=====================+===================+
| node-0_testing | ACTIVE   | 2025-01-28 01:45:13 | 0d 0h | 2025-01-28 01:54:55 | Flavor: m1.medium |
+----------------+----------+---------------------+-------+---------------------+-------------------+

NETWORKS to be deleted:
+--------------------+----------+---------------------+-------+---------------------+
| Name               | Status   | Created             | Age   | Last Seen           |
+====================+==========+=====================+=======+=====================+
| public_net_testing | ACTIVE   | 2025-01-28 01:45:07 | 0d 0h | 2025-01-28 01:54:55 |
+--------------------+----------+---------------------+-------+---------------------+
| exp_net0_

In [35]:
def release_floating_ips(server_data: Dict, dry_run: bool = True):
        """Release floating IPs associated with a server"""
        if not server_data.get('addresses'):
            return

        for network_name, addresses in server_data['addresses'].items():
            for addr in addresses:
                if addr.get('OS-EXT-IPS:type') == 'floating':
                    ip_address = addr.get('addr')
                    if ip_address:
                        if dry_run:
                            logger.info(f"Would release floating IP: {ip_address}")
                        else:
                            try:
                                floating_ips = list(os_conn.network.ips())
                                for ip in floating_ips:
                                    if ip.floating_ip_address == ip_address:
                                        os_conn.network.delete_ip(ip.id)
                                        logger.info(f"Released floating IP: {ip_address}")
                                        break
                            except Exception as e:
                                logger.error(f"Error releasing floating IP {ip_address}: {str(e)}")

In [24]:
def clean_router(router_data: Dict, dry_run: bool = True):
        """Clean router by removing external gateway and interfaces"""
        try:
            router_id = router_data['resource_id']
            if dry_run:
                if router_data.get('external_gateway_info'):
                    logger.info(f"Would remove external gateway from router: {router_id}")
                logger.info(f"Would remove interfaces from router: {router_id}")
                return

            # Get router ports
            ports = os_conn.network.ports(device_id=router_id)
            
            # Remove external gateway if exists
            if router_data.get('external_gateway_info'):
                os_conn.network.update_router(router_id, external_gateway_info={})
                logger.info(f"Removed external gateway from router: {router_id}")

            # Remove router interfaces
            for port in ports:
                if port.device_owner == 'network:router_interface':
                    os_conn.network.remove_interface_from_router(
                        router_id,
                        subnet_id=port.fixed_ips[0]['subnet_id']
                    )
            logger.info(f"Removed interfaces from router: {router_id}")

        except Exception as e:
            logger.error(f"Error cleaning router {router_id}: {str(e)}")
            raise

In [59]:
resources = get_resources_to_delete()

In [60]:
print(display_resources(resources))


SERVERS to be deleted:
+----------------+----------+---------------------+-------+---------------------+-------------------+
| Name           | Status   | Created             | Age   | Last Seen           | Details           |
+================+==========+=====================+=======+=====================+===================+
| node-0_testing | ACTIVE   | 2025-01-28 01:45:13 | 0d 0h | 2025-01-28 01:54:55 | Flavor: m1.medium |
+----------------+----------+---------------------+-------+---------------------+-------------------+

NETWORKS to be deleted:
+--------------------+----------+---------------------+-------+---------------------+
| Name               | Status   | Created             | Age   | Last Seen           |
+====================+==========+=====================+=======+=====================+
| public_net_testing | ACTIVE   | 2025-01-28 01:45:07 | 0d 0h | 2025-01-28 01:54:55 |
+--------------------+----------+---------------------+-------+---------------------+
| exp_net0_

In [62]:
for server in resources['servers']:
    try:
        server_id = server['resource_id']
        logger.info(f"Deleting server: {server['resource_name']} ({server['status']})")
                    
        # Release floating IPs first
        #release_floating_ips(server, dry_run=False)
                    
        # Delete the server
        os_conn.compute.delete_server(server_id)
        logger.info(f"Deleted server: {server_id}")
    except Exception as e:
        logger.error(f"Error deleting server {server_id}: {str(e)}")
        continue

2025-01-28 02:00:27,406 - __main__ - INFO - Deleting server: node-0_testing (ACTIVE)
2025-01-28 02:00:27,693 - __main__ - INFO - Deleted server: d3ba02e5-f582-4031-b769-fd19c3701d63


In [63]:
for router in resources['routers']:
    try:
        router_id = router['resource_id']
        logger.info(f"Processing router: {router['resource_name']} ({router['status']})")
                    
        # Clean router first
        clean_router(router, dry_run=False)
                    
        # Delete the router
        os_conn.network.delete_router(router_id)
        logger.info(f"Deleted router: {router_id}")
    except Exception as e:
        logger.error(f"Error deleting router {router_id}: {str(e)}")
        continue

2025-01-28 02:00:34,124 - __main__ - INFO - Processing router: inet_router_testing (ACTIVE)
2025-01-28 02:00:36,635 - __main__ - INFO - Removed external gateway from router: 382e2bbd-afad-4ad7-b6db-e21cc0b4ac98
2025-01-28 02:00:39,585 - __main__ - INFO - Removed interfaces from router: 382e2bbd-afad-4ad7-b6db-e21cc0b4ac98
2025-01-28 02:00:40,685 - __main__ - INFO - Deleted router: 382e2bbd-afad-4ad7-b6db-e21cc0b4ac98


In [64]:
for subnet in resources['subnets']:
    try:
        subnet_id = subnet['resource_id']
        logger.info(f"Deleting subnet: {subnet['resource_name']} ({subnet['cidr']})")
        os_conn.network.delete_subnet(subnet_id)
        logger.info(f"Deleted subnet: {subnet_id}")
    except Exception as e:
        logger.error(f"Error deleting subnet {subnet_id}: {str(e)}")
        continue

2025-01-28 02:00:43,603 - __main__ - INFO - Deleting subnet: public_subnet_testing (192.168.10.0/24)
2025-01-28 02:00:45,069 - __main__ - INFO - Deleted subnet: 414c2015-3412-4451-af14-ca67cae83512
2025-01-28 02:00:45,071 - __main__ - INFO - Deleting subnet: exp_subnet_net0_testing (192.168.1.0/24)
2025-01-28 02:00:47,076 - __main__ - INFO - Deleted subnet: f3a3f5f3-20c7-462f-9615-7db77757e1f6


In [65]:
for network in resources['networks']:
    try:
        network_id = network['resource_id']
        logger.info(f"Deleting network: {network['resource_name']} ({network['status']})")
        os_conn.network.delete_network(network_id)
        logger.info(f"Deleted network: {network_id}")
    except Exception as e:
        logger.error(f"Error deleting network {network_id}: {str(e)}")
        continue

2025-01-28 02:00:49,896 - __main__ - INFO - Deleting network: public_net_testing (ACTIVE)
2025-01-28 02:00:50,547 - __main__ - INFO - Deleted network: c0d7ee16-5054-456f-9353-09648250f997
2025-01-28 02:00:50,550 - __main__ - INFO - Deleting network: exp_net0_testing (ACTIVE)
2025-01-28 02:00:51,040 - __main__ - INFO - Deleted network: 2e0155fd-421d-45b5-ae79-0b367455a266
